# импорт библиотек

In [13]:
import numpy as np
import pandas as pd
import nltk
import re
import json
from pathlib import Path
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\маша\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\маша\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\маша\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# загрузка файлов в датафрейм

In [ ]:
def load_all_jsons(root_dir: str, include_long_sc: bool = False):
    root = Path(root_dir)
    patterns = ["main/**/*.json"]
    records = []
    for pat in patterns:
        for fp in root.glob(pat):
            try:
                with open(fp, "r", encoding="utf-8") as f:
                    data = json.load(f)
            except Exception as e:
                print(f"Ошибка чтения {fp}: {e}")
                continue
            for it in data:
                rec = dict(it)
                rec["_source_file"] = str(fp.relative_to(root))
                records.append(rec)
    return records

RE_WS = re.compile(r"\s+", flags=re.UNICODE)
def clean_text(s):
    if not isinstance(s, str):
        return ""
    s = s.strip()
    s = RE_WS.sub(" ", s)
    return s


In [46]:
records = load_all_jsons('data')
print("Загружено записей:", len(records))

df = pd.DataFrame(records)
for col in ["id","text","dataset","source","model","paraphrasing_type"]:
    if col not in df.columns:
        df[col] = None

# метки: human=0, ai & ai+rew = 1
df["label"] = df["source"].apply(lambda s: 0 if s == "human" else 1)

Загружено записей: 4317


In [47]:
X = df['text']
y = df['label']

# функция извлечения стилометрических признаков

In [48]:
def extract_stylo_features(text):
    sentences = sent_tokenize(text)
    words = word_tokenize(text)

    avg_sent_len = np.mean([len(word_tokenize(s)) for s in sentences]) if sentences else 0
    sent_len_var = np.var([len(word_tokenize(s)) for s in sentences]) if sentences else 0
    ttr = len(set(words)) / len(words) if words else 0
    markdown_bold = text.count("**")

    return pd.Series({
        "avg_sent_len": avg_sent_len,
        "sent_len_var": sent_len_var,
        "ttr": ttr,
        "markdown_bold": markdown_bold
    })

df_stylo = X.apply(extract_stylo_features)
df_features = pd.concat([df[['text']], df_stylo], axis=1)


# создание модели


In [51]:
tfidf = TfidfVectorizer(
    max_features=30000,
    ngram_range=(1,2),
    sublinear_tf=True
)

ct = ColumnTransformer([
    ("tfidf", tfidf, "text"),
    ("stylo", StandardScaler(), ["avg_sent_len", "sent_len_var", "ttr", "markdown_bold"]),
])

model = Pipeline([
    ('features', ct),
    ('clf', LogisticRegression(max_iter=400, class_weight="balanced"))
])


# обучение и оценка качества

In [52]:
X_train, X_test, y_train, y_test = train_test_split(df_features, y, test_size=0.2, random_state=42)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1-score:", f1_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.8206018518518519
F1-score: 0.8541862652869238
              precision    recall  f1-score   support

           0       0.65      0.93      0.77       273
           1       0.96      0.77      0.85       591

    accuracy                           0.82       864
   macro avg       0.81      0.85      0.81       864
weighted avg       0.86      0.82      0.83       864



In [53]:
feature_names = model.named_steps['features'].named_transformers_['tfidf'].get_feature_names_out()
coefs = model.named_steps['clf'].coef_[0][:len(feature_names)]

top_ai = np.argsort(coefs)[-20:][::-1]  
top_human = np.argsort(coefs)[:20]      

print("\n🔹 Маркеры AI:")
for i in top_ai:
    print(feature_names[i], round(coefs[i], 4))

print("\n🔹 Маркеры Human:")
for i in top_human:
    print(feature_names[i], round(coefs[i], 4))



🔹 Маркеры AI:
однако 1.7649
включая 1.6421
её 1.5101
информирует 1.4952
этот 1.4837
для 1.4335
не только 1.2585
этом информирует 1.2147
как 1.1527
особенно 1.1425
несмотря на 1.1242
несмотря 1.1023
важно 1.0976
демонстрирует 1.0758
подчеркивает 1.0614
вопрос 1.0315
это 1.0279
эти 1.007
ссылаясь 0.9596
ссылаясь на 0.9596

🔹 Маркеры Human:
сообщает -2.1748
этом сообщает -1.6725
об -1.6064
очень -1.4899
со ссылкой -1.4605
ссылкой -1.4431
ссылкой на -1.4153
об этом -1.4138
числе -1.3866
пишет -1.3734
том числе -1.3657
потому -1.3353
проблему -1.2053
сказал -1.1995
потому что -1.1925
со -1.1635
по -1.1587
гг -1.1545
сообщается -1.1541
на сайте -1.1127


In [54]:
stylo_importance = model.named_steps['clf'].coef_[0][-4:]
stylo_feats = ["avg_sent_len","sent_len_var","ttr","markdown_bold"]

print("Стилометрия (влияние на класс AI):")
for f, w in zip(stylo_feats, stylo_importance):
    print(f"{f}: {round(w, 4)}")


Стилометрия (влияние на класс AI):
avg_sent_len: 0.4925
sent_len_var: -0.8599
ttr: 0.2969
markdown_bold: 6.9352


# функция проверки модели

In [56]:
label_map = {1: "AI-generated", 0: "Human-written"}

def classify_text(text: str):
    sample = pd.DataFrame([{
        "text": text,
        **extract_stylo_features(text)
    }])
    pred = model.predict(sample)[0]
    proba = model.predict_proba(sample)[0][1]
    label = label_map[pred]
    confidence = proba if pred == 1 else 1 - proba
    return {"prediction": label, "confidence": float(confidence)}

# пример
example = '''

**Москва, 15 марта 2024 г.** — Группа исследователей из Национального исследовательского центра «Курчатовский институт» и МИФИ представила прототип квантового компьютера, ключевые компоненты которого — кубиты и управляющая электроника — полностью разработаны и произведены в России.

Устройство, получившее название «Квант-1», пока обладает скромной мощностью в 8 кубитов, но его архитектура позволяет масштабировать систему до сотен кубитов в ближайшие три года. Главным прорывом ученые называют создание собственной технологии производства сверхпроводящих кубитов на основе алюминиевых структур, которые способны сохранять квантовое состояние рекордное для отечественных разработок время — более 100 микросекунд.

«Это не просто лабораторный макет. Мы построили полный технологический цикл: от моделирования и производства чипов до системы управления и программного стека. Это фундамент для будущего полноценного квантового процессора», — заявил руководитель проекта, академик Иван Петров.

Разработка велась в рамках национального проекта «Наука и университеты». Первыми практическими применениями «Кванта-1» станут задачи оптимизации логистики и разработки новых материалов. До конца года к системе получат доступ исследователи из нескольких российских университетов и компаний-партнеров.

Эксперты отмечают, что, несмотря на отставание от лидеров в отрасли (компаний Google и IBM), создание полностью отечественной платформы является критически важным шагом для технологического суверенитета и безопасности страны в долгосрочной перспективе.'''
print(classify_text(example))


{'prediction': 'AI-generated', 'confidence': 0.8843162495168472}


In [57]:
import joblib
joblib.dump(model, "model.pkl")

['model.pkl']